In [3]:
import numpy as np
import baseline_model
from casadi import *
import do_mpc
import decentralized as dec


In [6]:
Q = np.eye(6)
R = np.eye(3)
n_agents = 3
n_inputs = 3
n_states = 6
Qf = Q*1e3

In [7]:
Qs = [Q] * n_agents
Rs = [R] * n_agents
Qfs = [Qf] * n_agents

In [8]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

x = model.set_variable(var_type='_x', var_name='x', shape=(6, 1))
u = model.set_variable(var_type='_u', var_name='u', shape=(3, 1))
print(f'Shape of x is {x.shape}')
print(f'Shape of u is {u.shape}')
#x = p_x,p_y,p_z,v_x,v_y,v_z
#u = theta, phi, tau
g = 9.81
model.set_rhs('x', vertcat(x[3], x[4], x[5], g*np.tan(u[0]), -g*np.tan(u[1]), u[2]-g))

Shape of x is (6, 1)
Shape of u is (3, 1)


In [13]:
xf = np.random.rand(18)
x_dims = [6,6,6]

In [41]:


total_stage_cost= [(x-xf_i.T).T@Qi@(x-xf_i.T) + u.T@Ri@u
    for xf_i, Qi, Ri, in zip(
        dec.split_agents(xf.reshape(1,-1), x_dims), Qs, Rs, 
    )]
 

total_terminal_cost = [(x-xf_i.T).T@Qfi@(x-xf_i.T)
#shape is not correct unless xf_i is transposed
for xf_i, Qfi in zip(
    dec.split_agents(xf.reshape(1,-1), x_dims), Qfs
)]

In [42]:
len(total_stage_cost)

3

In [50]:
model.set_expression('total_stage_cost',np.sum(total_stage_cost))
model.set_expression('total_terminal_cost',np.sum(total_terminal_cost))

SX(@1=1000, @2=0.67056, @3=0.3322, @4=0.589424, @5=0.59735, @6=0.0866769, @7=0.324466, @8=0.0406781, @9=0.0136367, @10=0.929077, @11=0.449056, @12=0.608274, @13=0.800826, @14=0.637524, @15=0.414727, @16=0.184916, @17=0.614452, @18=0.838018, @19=0.657215, (((((((((@1*(x_0-@2))*(x_0-@2))+((@1*(x_1-@3))*(x_1-@3)))+((@1*(x_2-@4))*(x_2-@4)))+((@1*(x_3-@5))*(x_3-@5)))+((@1*(x_4-@6))*(x_4-@6)))+((@1*(x_5-@7))*(x_5-@7)))+(((((((@1*(x_0-@8))*(x_0-@8))+((@1*(x_1-@9))*(x_1-@9)))+((@1*(x_2-@10))*(x_2-@10)))+((@1*(x_3-@11))*(x_3-@11)))+((@1*(x_4-@12))*(x_4-@12)))+((@1*(x_5-@13))*(x_5-@13))))+(((((((@1*(x_0-@14))*(x_0-@14))+((@1*(x_1-@15))*(x_1-@15)))+((@1*(x_2-@16))*(x_2-@16)))+((@1*(x_3-@17))*(x_3-@17)))+((@1*(x_4-@18))*(x_4-@18)))+((@1*(x_5-@19))*(x_5-@19)))))

In [ ]:
|